In [2]:
from transformers import AutoTokenizer
from llama_zh.modeling_llama_zh import LlamaForCausalLM
import torch
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList, GenerationConfig, ModelOutput

from typing import List, Tuple
class InvalidScoreLogitsProcessor(LogitsProcessor):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        if torch.isnan(scores).any() or torch.isinf(scores).any():
            scores.zero_()
            scores[..., 5] = 5e4
        return scores

@torch.no_grad()
def chat(model, tokenizer, query: str, history: List[Tuple[str, str]] = None, max_length: int = 2048, num_beams=1,
            do_sample=True, top_p=0.7, temperature=0.95, logits_processor=None, **kwargs):
    if history is None:
        history = []
    if logits_processor is None:
        logits_processor = LogitsProcessorList()
    logits_processor.append(InvalidScoreLogitsProcessor())
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "do_sample": do_sample, "top_p": top_p,
                    "temperature": temperature, "logits_processor": logits_processor, **kwargs}
    if not history:
        prompt = query
    else:
        prompt = ""
        for i, (old_query, response) in enumerate(history):
            prompt += "[Round {}]\n问：{}\n答：{}\n".format(i, old_query, response)
        prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)
    inputs = tokenizer([prompt], return_tensors="pt")
    inputs = inputs.to(model.device)
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):]
    response = tokenizer.decode(outputs)
    # response = model.process_response(response)
    history = history + [(query, response)]
    return response, history


# tokenizer = AutoTokenizer.from_pretrained("output/llama_zh_001/checkpoint-10/", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
        "llama_zh", trust_remote_code=True) 
model = LlamaForCausalLM.from_pretrained("output/llama_zh_001/checkpoint-40").cuda()
response, history = chat(model=model, tokenizer=tokenizer, query="你好啊")

print(response)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\yuanz\AppData\Local\Temp/ipykernel_6868/3847226611.py:42 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\yuanz\\AppData\\Local\\Temp/ipykernel_6868/3847226611.py'                            │
│                                                                                                  │
│ c:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\transformers\models\auto\tokenization_auto │
│ .py:686 in from_pretrained                                                                       │
│                                                                                                  │
│   683 │   │   │   │   │   class_ref = tokenizer_auto_map[0]                                      │
│   684 │   │   │   │                                                                              │
│   685 │   │   │   │   module_file, class_name = class_ref.split(".")                             │
│ ❱ 686 │   │   │   │   tokenizer_class = get_class_from_dynamic_module(                           │
│   687 │   │   │   │   │   pretrained_model_name_or_path, module_file + ".py", class_name, **kw   │
│   688 │   │   │   │   )                                                                          │
│   689 │   │   │   │   tokenizer_class.register_for_auto_class()                                  │
│                                                                                                  │
│ c:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\transformers\dynamic_module_utils.py:388   │
│ in get_class_from_dynamic_module                                                                 │
│                                                                                                  │
│   385 │   │   revision=revision,                                                                 │
│   386 │   │   local_files_only=local_files_only,                                                 │
│   387 │   )                                                                                      │
│ ❱ 388 │   return get_class_in_module(class_name, final_module.replace(".py", ""))                │
│   389                                                                                            │
│   390                                                                                            │
│   391 def custom_object_save(obj, folder, config=None):                                          │
│                                                                                                  │
│ c:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\transformers\dynamic_module_utils.py:157   │
│ in get_class_in_module                                                                           │
│                                                                                                  │
│   154 │   Import a module on the cache directory for modules and extract a class from it.        │
│   155 │   """                                                                                    │
│   156 │   module_path = module_path.replace(os.path.sep, ".")                                    │
│ ❱ 157 │   module = importlib.import_module(module_path)                                          │
│   158 │   return getattr(module, class_name)                                                     │
│   159                                                                                            │
│   160                                                                                            │
│                                                                                                  │
│ c:\Users\yuanz\anaconda3\envs\mynet\lib\importlib\__init__.